# TREC 2018 Precision Medicine

In [1]:
import json
from json2html import *
from IPython.display import HTML
import eval

In [2]:
with open('config.json', 'r') as f:
    config = json.load(f)
#HTML(json2html.convert(json = config))

Load TOPICS and GOLD STANDARD from last year (30 topics)

In [3]:
topics_df = eval.get_topics('./topics/topics2017.xml')

In [4]:
#topics_df

In [5]:
qrels = eval.get_qrels('./gold-standard/sorted.abstracts.2017.qrels')

In [6]:
eval.experiment(topics_df, qrels)

{'gene_tie_breaker': [0.1, 0.5], 'gene_boost': [1, 2, 5], 'disease_boost': [1, 2, 5], 'disease_tie_breaker': [0.1, 0.5], 'query_template': ['variable.json']}
RUN: FIXME {'gene_tie_breaker': '0.1', 'gene_boost': '1', 'disease_boost': '1', 'disease_tie_breaker': '0.1', 'query_template': 'variable.json'}
{'P_10': 0.3067, 'ndcg': 0.4139, 'Rprec': 0.1952}
RUN: FIXME {'gene_tie_breaker': '0.1', 'gene_boost': '2', 'disease_boost': '1', 'disease_tie_breaker': '0.1', 'query_template': 'variable.json'}
{'P_10': 0.3033, 'ndcg': 0.4003, 'Rprec': 0.1852}
RUN: FIXME {'gene_tie_breaker': '0.1', 'gene_boost': '5', 'disease_boost': '1', 'disease_tie_breaker': '0.1', 'query_template': 'variable.json'}
{'P_10': 0.2933, 'ndcg': 0.3936, 'Rprec': 0.181}
RUN: FIXME {'gene_tie_breaker': '0.5', 'gene_boost': '1', 'disease_boost': '1', 'disease_tie_breaker': '0.1', 'query_template': 'variable.json'}
{'P_10': 0.3, 'ndcg': 0.3968, 'Rprec': 0.1853}
RUN: FIXME {'gene_tie_breaker': '0.5', 'gene_boost': '2', 'disease

In [7]:
run_df = eval.run(topics_df)

RUN: FIXME {'gene_tie_breaker': 0.5, 'gene_boost': 1, 'disease_boost': 1.5, 'disease_tie_breaker': 0.5, 'query_template': 'baseline.json'}


In [8]:
#run_df

In [9]:
results, aggregated = eval.evaluate(qrels, run_df)

In [10]:
aggregated

{'P_10': 0.48, 'Rprec': 0.2847, 'ndcg': 0.5338}